In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
import xgboost as xgb
import matplotlib
import pickle

In [87]:
train = pd.read_csv('Train_maskedv2.csv')
test = pd.read_csv('Test_maskedv2.csv')
submissions = pd.read_csv('samplesubmissionv2.csv')
vd = pd.read_csv('variable_descriptions_v2.csv')

In [88]:
print(train.shape)
print(test.shape)

(3174, 50)
(1102, 49)


In [89]:
#train1 = pd.read_csv('Train_maskedv2.csv')
Labels = np.array(train['target_pct_vunerable'])
print(Labels)

[1.73980135 1.99338128 0.03834043 ... 0.95882823 1.02493356 5.28312496]


In [90]:
train = train.drop(['ward'], axis = 1)
test = test.drop(['ward'], axis = 1)

In [94]:
#excluded_cols = ['dw_03', 'dw_05', 'pg_02', 'lan_14', 'psa_03', 'lan_02', 'dw_02', 'dw_10', 
               # 'psa_02', 'lan_07', 'lan_09', 'lan_11', 'dw_11', 'lan_10']
    
excluded_cols = ['dw_12', 'dw_13', 'lan_13', 'dw_10', 'psa_02', 'lan_07', 'lan_09']
for i in excluded_cols:
    test = test.drop(i, axis=1)
    
for j in excluded_cols:
    train = train.drop(j, axis=1)

In [91]:
train = train.drop(['target_pct_vunerable'], axis=1)
print(train.shape)
print(test.shape)

(3174, 48)
(1102, 48)


In [92]:
train['avg_individuals'] = train['total_individuals'] / train['total_households']
test['avg_individuals'] = test['total_individuals'] / test['total_households']

In [93]:
train['psa_234'] = train['psa_02'] + train['psa_03'] + train['psa_04']

In [12]:
test['psa_234'] = test['psa_02'] + test['psa_03'] + test['psa_04']

In [95]:
Features = np.array(train)
test_enc = np.array(test)

In [96]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(Features[:, :2])
Features[:, :2] = scaler.transform(Features[:, :2])
print(Features[:, :2])

[[ 3.25349424  2.85925741]
 [ 3.2611637   2.1992301 ]
 [-0.23453747 -0.44538003]
 ...
 [ 0.93342628  1.8316802 ]
 [ 3.1076322   3.35482504]
 [ 0.25944328  0.27111737]]


In [97]:
scaler1 = preprocessing.StandardScaler().fit(test_enc[:, :2])
test_enc[:, :2] = scaler1.transform(test_enc[:, :2])
print(test_enc[:, :2])

[[-0.16503446 -0.35938299]
 [-0.42163757 -0.42024847]
 [-0.46203328 -0.6051864 ]
 ...
 [ 0.32568337  0.19777147]
 [-0.32950555 -0.10753076]
 [-0.38886785 -0.5674457 ]]


In [98]:
import numpy.random as nr
import sklearn.model_selection as ms
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [99]:
indx1 = range(x_train.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.2)
x_train1 = x_train[indx1[0],:]
y_train1 = np.ravel(y_train[indx1[0]])
x_train_val = x_train[indx1[1],:]
y_train_val = np.ravel(y_train[indx1[1]])

In [100]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=150, max_depth=9, max_features='auto',
                            min_samples_leaf=1, min_samples_split=2, random_state=0)
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [101]:
rf_pred = rf.predict(x_test)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mean_squared_error(rf_pred, y_test) ** 0.5

6.758065000758289

In [85]:
preds = rf.predict(test_enc)
submissions.target_pct_vunerable = preds
submissions.to_csv('SouthAfricaCovid5.csv', index=False)

In [103]:
import lgbm

ModuleNotFoundError: No module named 'lgbm'